In [8]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

class GroupDataSharingSystem:
    def _init_(self):
        self.group_members = {}  # Stores users' RSA keys and encrypted AES keys
        self.aes_key = None      # AES key for encrypting data

    def generate_rsa_keys(self):
        """
        Generate RSA public and private keys for a user.
        """
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
            backend=default_backend()
        )
        public_key = private_key.public_key()
        return private_key, public_key

    def add_member(self, identity):
        """
        Add a new member to the group with their own RSA key pair and an encrypted AES key.
        """
        private_key, public_key = self.generate_rsa_keys()

        # Encrypt AES key for this user using their public RSA key
        encrypted_aes_key = public_key.encrypt(
            self.aes_key,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )

        # Store the member's information
        self.group_members[identity] = {
            "private_key": private_key,
            "public_key": public_key,
            "encrypted_aes_key": encrypted_aes_key
        }
        print(f"Member {identity} added to the group with RSA encryption.")

    def generate_aes_key(self):
        """
        Generate a symmetric AES key for data encryption.
        """
        self.aes_key = os.urandom(32)  # AES-256 key
        print("AES key generated for data encryption.")

    def encrypt_data(self, data):
        """
        Encrypt data using the AES key.
        """
        iv = os.urandom(16)  # Initialization vector for AES
        cipher = Cipher(algorithms.AES(self.aes_key), modes.CFB(iv), backend=default_backend())
        encryptor = cipher.encryptor()
        encrypted_data = iv + encryptor.update(data.encode()) + encryptor.finalize()
        print("Data encrypted using AES key.")
        return encrypted_data

    def decrypt_data(self, encrypted_data, identity):
        """
        Decrypt data for a specific member by decrypting the AES key with their RSA private key.
        """
        member_info = self.group_members.get(identity)

        if not member_info:
            return f"User {identity} is not a member of the group."

        private_key = member_info["private_key"]

        # Decrypt the AES key using the user's private RSA key
        aes_key = private_key.decrypt(
            member_info["encrypted_aes_key"],
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )

        # Decrypt the data using the decrypted AES key
        iv = encrypted_data[:16]  # Extract IV
        cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv), backend=default_backend())
        decryptor = cipher.decryptor()
        decrypted_data = decryptor.update(encrypted_data[16:]) + decryptor.finalize()
        return decrypted_data.decode()

    def remove_member(self, identity):
        """
        Remove a member from the group.
        """
        if identity in self.group_members:
            del self.group_members[identity]
            print(f"Member {identity} removed from the group.")
        else:
            print(f"Member {identity} not found in the group.")
